# Амортизационный анализ

## Aггрегационный метод

Рассмотрим классическую реализацию динамического массива. Нам для целей примера нужна только операция push_back

In [7]:
class SimpleDynamicArray:
    def __init__(self, capacity=1):
        self.capacity = capacity
        self.size = 0
        self.array = [None] * capacity

    def push_back(self, element):
        # Когда capacity заканчивается, создаем новый массив вдвое большей длины
        if self.size == self.capacity:
            new_capacity = self.capacity * 2
            new_array = [None] * new_capacity

            for i in range(self.size):
                new_array[i] = self.array[i]

            self.array = new_array
            self.capacity = new_capacity

        self.array[self.size] = element
        self.size += 1

Заметим что, если в данный момент size < capacity, то push_back нового элемента занимает $O(1)$. Однако если size = capacity, то push_back делается уже за целых $O(n)$.

То есть, строго говоря, асимптотика функции push_back в общем - $O(n)$. Но мы же понимаем, что при многократных запросах на добавление ***сильно чаще*** триггерится именно первый вариант поведения с $O(1)$, из-за того, что capacity растет экспоненциально.

Это можно воспринимать так - нам важно не то, сколько выполняется каждая операция **по отдельности**, зато важно сколько выполняется **вся цепочка** из $n$ операций. Если смотреть на пример с динамическим массивом так, то действительно цена выполнения цепочки из $n$ операций:

$T_{\text{all}} = O(\sum_1^{n} 1 + \sum_{j = 1}^{\lfloor log(n - 1) \rfloor} 2^j) = O(n + 2^{\lfloor log(n - 1) \rfloor}) = O(n + n - 1) = O(n)$

Таким образом средняя цена выполнения каждой из $n$ операций:

$T_{\text{avg}} = \frac{T_{\text{all}}}{n} = O(1)$ - это и есть амортизированная асимптотика, а такой простейший подход к амортизационному анализу называется Аггрегационным методом

Амортизационный анализ позволяет формализовать эти мысли в виде понятия амортизированного времени и предоставляет инструменты подсчета его асимптотики

## Метод кредитов

Но сначала рассмотрим еще один пример, который вплотную подведет нас к формальным определениям.

Попробуем сделать очередь с операцией гета минимума за $O(1)$.

Для этого для начала сделаем стек с гетом минимума за O(1), как промежуточный шаг. Идея очень простая, заведем два стека, в одном, обычном, будем хранить реальные значения, а во втором минимумы на подотрезках [0, k] обычного стека.

In [11]:
class MinStack:
    def __init__(self):
        self.stack = []  # Обычный стек для реальных значений
        self.min_stack = []  # Стек минимумом на подотрезках [0, k] в обычном стеке

    def push(self, value):
        self.stack.append(value)
        if not self.min_stack or value <= self.min_stack[-1]: # Вся реальная новая логика здесь
            self.min_stack.append(value)

    def pop(self):
        if not self.stack:
            return None
        self.min_stack.pop()
        return self.stack.pop()

    def top(self):
        if not self.stack:
            return None
        return self.stack[-1]

    def get_min(self):
        if not self.min_stack:
            return None
        return self.min_stack[-1]

Все с этим стеком честно и хорошо, все операции выполняются за честные $O(1)$ (если не считать того, что под капотом у наших стеков динамический массив с амортизированным O(1), но забудем об этом)

С MinQueue всё уже не так хорошо. Его будем делать так:
1. Разрежем Queue пополам на две части
2. Сделаем обе части MinStack'ами
3. Если нам нужно будет добавить элемент в очередь, то будем добавлять его в правый MinStack
4. Если нам нужно будет посчитать минимум, будем считать его на левой и правой части, и брать минимум из результатов
5. Если нам нужно будет забирать элементы из очереди, то будем делать pop у левого MinStack. Если он пустой, то сначала циклически переложим элементы из правого MinStack в левый

In [ ]:
class MinQueue:
    def __init__(self):
        self.left_stack = MinStack()
        self.right_stack = MinStack()

    def push(self, value):
        self.right_stack.push(value)

    def pop_front(self):
        if not self.left_stack.stack:
            while self.right_stack.stack:
                self.left_stack.push(self.right_stack.pop())
        return self.left_stack.pop()

    def get_min(self):
        left_min = self.left_stack.get_min()
        right_min = self.right_stack.get_min()

        if left_min is None:
            return right_min
        if right_min is None:
            return left_min

        return min(left_min, right_min)

Здесь уже начинаются проблемы. Если left_stack не пустой, то pop_front отрабатывает за $O(1)$, иначе - $O(n)$. Ситуация похожа на динамический массив, но есть и отличия - в примере с динамическим массивом проблема возникала с операцией добавления, а здесь с операцией получения. Аггрегационный метод здесь уже будет неудобно применять.

Представим, что при добавлении элемента в MinQueue мы выдаем ему 2 монетки, которыми он оплачивает операции, меняющие стейт. Тогда в худшем случае получится,что одну монетку он платит за то, что его переложат из right_stack в left_stack и одну за то, что его достанут из left_stack с помощью pop_front.

Таким образом, **при любой** последовательности операций, средняя стоимость по всей последовательности будет $O(1)$. Такой подход и называется методом кредитов.

Попробуем наконец формализовать все эти наблюдения

## Формальные определения

Пусть имеется структура данных $D$ c набором состояний $S = \{ s_0, s_1, s_2, \dots \}$, поддерживающая семейство операций $\mathcal{O}=\{o_1,o_2,\dots\}$.

Для любой операции $o$ в состоянии $s$ определена её **фактическая цена выполнения** $T(o, s)\in\mathbb{R}_{\ge0}$.

Рассмотрим последовательность операций $\sigma=(o_1,o_2,\dots,o_n)$, выполняемую на начальном состоянии $s_0$. Через $s_i$ обозначим состояние после выполнения первых $i$ операций. Тогда суммарная фактическая стоимость очевидно равна $T(\sigma) \;=\;\sum_{i=1}^n T\bigl(o_i, s_{i-1}\bigr).
$

***Определение:*** Амортизационная стоимость $\hat{T}(o_i, s_{i-1})$ - условная стоимость операции $o_i$, выбранная так, чтобы $\forall \sigma$ - последовательность операций выполнялось

$$
\hat{T}(\sigma) = \sum_{i=1}^n \hat{T}(o_i, s_{i-1})\; \ge\;T(\sigma),
$$

Тогда **амортизационная стоимость на операцию** $\hat{T}(o_i) = \max_{s \in S}\hat{T}(o_i, s)$

Теперь с помощью этого давайте формализуем метод кредитов на примере MinQueue

Набор операций $\mathcal{O} = \{ \text{push\_back}, \text{pop\_front} \}$. get_min не будем рассматривать, тк это немутирующая операция

Набор состояний $S = \{ \text{left\_stack\_empty}, \text{left\_stack\_filled} \}$.

Начальное состояние $s_0$

Посчитаем обычные функции стоимости на операциях из разных состояний:

$T(\text{push\_back}, \text{left\_stack\_empty}) = T(\text{push\_back}, \text{left\_stack\_filled}) = O(1)$

$T(\text{pop\_front}, \text{left\_stack\_empty}) = O(n)$

$T(\text{pop\_front}, \text{left\_stack\_filled}) = O(1)$

Теперь введем амортизированные стоимости с помощью метода кредитов:

$\hat{T}(\text{push\_back}, \text{left\_stack\_empty}) = \hat{T}(\text{push\_back}, \text{left\_stack\_filled}) = T(\text{push\_back}, -) + 2 * C$ - выдаем кредит ($C$ - константа "номинал" монетки, понадобится в следующем шаге)

$\hat{T}(\text{pop\_front}, \text{left\_stack\_empty}) = T(\text{pop\_front}, \text{left\_stack\_empty}) - C * n$ - забираем кредит ($C$ нужно, чтобы убить компоненту $n$ в $O(n)$ и оставить итоговое выражение константой)

$\hat{T}(\text{pop\_front}, \text{left\_stack\_filled}) = T(\text{pop\_front}, \text{left\_stack\_filled}) - C$ - забираем кредит

Очевидно, что  $\forall \sigma$ $\hat{T}(\sigma) = T(\sigma)$, а значит условие на амортизированное время выполнено. Таким образом амортизированное время операций:

$\hat{T}(\text{push\_back}) = O(1) + 2 * C = O(1)$

$\hat{T}(\text{push\_back}) = T(\text{pop\_front}, \text{left\_stack\_empty}) - C * n = O(1)$ - так константа $C$ введена


Рассмотрим еще один пример - HeapSet. Эта структура данных содержит набор куч $A_1, ..., A_k$ и поддерживает операции:
1. add(A_i, x) - добавить новый элемент $x$ в кучу $A_i$
2. merge(A_i, A_j) - слить кучи $A_i$ и $A_j$ в одну
3. get_min(A_i) - получить минимум на $i$-той куче

Напишем сначала стандартную кучу:

In [1]:
class MinHeap:
    def __init__(self):
        self.heap = []

    def parent(self, i):
        return (i - 1) // 2

    def left(self, i):
        return 2 * i + 1

    def right(self, i):
        return 2 * i + 2

    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def sift_up(self, i):
        parent = self.parent(i)
        if i > 0 and self.heap[i] < self.heap[parent]:
            self.swap(i, parent)
            self.sift_up(parent)

    def sift_down(self, i):
        min_idx = i
        left = self.left(i)
        right = self.right(i)

        if left < len(self.heap) and self.heap[left] < self.heap[min_idx]:
            min_idx = left

        if right < len(self.heap) and self.heap[right] < self.heap[min_idx]:
            min_idx = right

        if min_idx != i:
            self.swap(i, min_idx)
            self.sift_down(min_idx)

    def add(self, key):
        self.heap.append(key)
        self.sift_up(len(self.heap) - 1)

    def get_min(self):
        if not self.heap:
            return None
        return self.heap[0]

Надеюсь ничего нового не было, напишем теперь собственно HeapSet. Нам осталось написать только функцию merge - она будет приливать меньшую кучу из двух к большей с помощью операции add

In [3]:
class HeapSet:
    def __init__(self, heaps):
        self.heaps = heaps

    def merge(self, i, j):
        large_heap = self.heaps[i] if len(self.heaps[i].heap) >= len(self.heaps[j].heap) else self.heaps[j]
        small_heap = self.heaps[j] if len(self.heaps[i].heap) > len(self.heaps[j].heap) else self.heaps[i]

        # Приливаем меньшую кучу к большей
        while small_heap.heap:
            large_heap.add(small_heap.get_min())
            small_heap.heap.pop(0)

Очевидно, каждый отдельный мердж, происходит за $O(|A_j| * log(|A_i| + |A_j|))$, где $A_j$ - меньшая куча

Наша задача показать, что все мерджи вместе происходят всего за $O((n + q)*log^2(n + q))$, где $n$ - суммарный размер всех куч, $q$ - число операций

**Теорема:** Амортизированная сложность всех операций merge в HeapSet составляет $O((n + q) \cdot \log^2(n + q))$.

**Доказательство:**

Введем схему кредитования: при добавлении элемента в кучу $A_i$ мы выдаем ему $c \cdot \log^2(n + q_{\text{current}})$ монет, где $c$ - некоторая константа, а $q_{\text{current}}$ - количество операций, выполненных к текущему моменту.

Рассмотрим операцию merge$(A_i, A_j)$, где без ограничения общности $|A_i| \geq |A_j|$. Фактическая стоимость этой операции:

$$T(\text{merge}, A_i, A_j) = O(|A_j| \cdot \log(|A_i| + |A_j|))$$

Это происходит потому, что мы добавляем каждый из $|A_j|$ элементов в кучу размера не более $|A_i| + |A_j|$, и каждое добавление стоит $O(\log(|A_i| + |A_j|))$.

Теперь проанализируем, сколько монет накопилось у элементов кучи $A_j$. Каждый элемент получил $c \cdot \log^2(n + q')$ монет при добавлении, где $q'$ - количество операций на момент добавления элемента. Поскольку $q' \leq q$, каждый элемент имеет не менее $c \cdot \log^2(n)$ монет.

Общее количество монет у всех элементов кучи $A_j$ составляет не менее:

$$|A_j| \cdot c \cdot \log^2(n) \geq c \cdot |A_j| \cdot \log^2(|A_i| + |A_j|)$$

При достаточно большой константе $c$ этих монет хватит для оплаты фактической стоимости операции merge.

Таким образом, амортизированная стоимость всех операций составляет:
- Для add: $O(1) + c \cdot \log^2(n + q) = O(\log^2(n + q))$
- Для merge: $O(1)$ (так как оплата происходит из накопленных монет)

Суммарная амортизированная стоимость всех операций:
$$O((n + q) \cdot \log^2(n + q))$$

Что и требовалось доказать.

### Проблема с операцией delete в HeapSet

Если мы добавим операцию delete в наш HeapSet, то предыдущий анализ амортизированной сложности перестает работать. Рассмотрим, почему это происходит.

**Теорема:** При добавлении операции delete в HeapSet, метод кредитов с предыдущей схемой кредитования не гарантирует амортизированную сложность $O((n + q) \cdot \log^2(n + q))$.

**Доказательство:**

Предположим, что мы добавили операцию delete$(A_i, x)$, которая удаляет элемент $x$ из кучи $A_i$. Фактическая стоимость этой операции составляет $O(\log |A_i|)$.

Проблема возникает в следующем сценарии:
1. Добавляем элементы в кучи $A_i$ и $A_j$, выдавая каждому элементу кредиты
2. Выполняем merge$(A_i, A_j)$, используя кредиты элементов кучи $A_j$
3. Удаляем все элементы, которые изначально были в куче $A_j$, с помощью операции delete

После этих операций мы потеряли все монеты, которые были выданы элементам кучи $A_j$, но эти элементы больше не находятся в структуре данных. Если мы снова добавим элементы в новую кучу $A_k$ и выполним merge$(A_i, A_k)$, то у элементов кучи $A_k$ не будет достаточно монет для оплаты этой операции, так как размер кучи $A_i$ мог значительно вырасти из-за предыдущего слияния.

Таким образом, наша схема кредитования перестает работать.

### Решение: HeapSet с рангами

Для решения этой проблемы можно использовать модификацию HeapSet с рангами, аналогично тому, как это делается в структуре данных DSU (Disjoint Set Union) с оптимизацией Union by Rank.

```python
class RankedMinHeap(MinHeap):
    def __init__(self):
        super().__init__()
        self.rank = 0  # Ранг кучи - логарифм от её размера

    def add(self, key):
        super().add(key)
        # Обновляем ранг при необходимости
        self.rank = max(self.rank, int(math.log2(len(self.heap) + 1)))

class RankedHeapSet:
    def __init__(self, heaps):
        self.heaps = heaps

    def merge(self, i, j):
        # Всегда объединяем кучу с меньшим рангом с кучей с большим рангом
        if self.heaps[i].rank < self.heaps[j].rank:
            i, j = j, i

        large_heap = self.heaps[i]
        small_heap = self.heaps[j]

        # Приливаем меньшую кучу к большей
        while small_heap.heap:
            large_heap.add(small_heap.get_min())
            small_heap.heap.pop(0)

        # Обновляем ранг результирующей кучи
        large_heap.rank = max(large_heap.rank, small_heap.rank + 1)

    def delete(self, i, x):
        # Реализация удаления элемента x из кучи i
        # ...
```

**Теорема:** С использованием рангов, амортизированная сложность всех операций (включая delete) в HeapSet составляет $O((n + q) \cdot \log^2(n + q))$.

**Доказательство:**

При использовании рангов мы гарантируем, что при слиянии куч размер результирующей кучи как минимум вдвое больше размера меньшей кучи. Это означает, что элемент может участвовать в операции merge в качестве элемента меньшей кучи не более $\log n$ раз за всё время работы структуры.

Таким образом, если мы выдаем каждому элементу $c \cdot \log^3(n + q)$ монет при добавлении, этих монет хватит для оплаты всех операций merge, в которых этот элемент может участвовать, даже если другие элементы будут удалены.

Амортизированная стоимость операций:
- add: $O(1) + c \cdot \log^3(n + q) = O(\log^3(n + q))$
- merge: $O(1)$ (оплачивается из накопленных монет)
- delete: $O(\log n) = O(\log(n + q))$

Суммарная амортизированная стоимость всех операций:
$$O((n + q) \cdot \log^3(n + q))$$

Это немного хуже, чем в случае без операции delete, но всё равно является полиномиальной оценкой от $\log(n + q)$.

## Пример дека с минимумом, на котором метод кредитов не работает

Рассмотрим структуру данных MinDeque, которая поддерживает следующие операции:
1. push_front(x) - добавить элемент x в начало дека
2. push_back(x) - добавить элемент x в конец дека
3. pop_front() - удалить элемент из начала дека
4. pop_back() - удалить элемент из конца дека
5. get_min() - получить минимальный элемент в деке

Одна из возможных реализаций MinDeque - использование двух MinStack:

```python
class MinDeque:
    def __init__(self):
        self.front_stack = MinStack()  # Стек для элементов в начале дека
        self.back_stack = MinStack()   # Стек для элементов в конце дека (в обратном порядке)

    def push_front(self, x):
        self.front_stack.push(x)

    def push_back(self, x):
        self.back_stack.push(x)

    def pop_front(self):
        if not self.front_stack.stack:
            # Если front_stack пуст, перекладываем элементы из back_stack
            self._rebalance()
        return self.front_stack.pop()

    def pop_back(self):
        if not self.back_stack.stack:
            # Если back_stack пуст, перекладываем элементы из front_stack
            self._rebalance_reverse()
        return self.back_stack.pop()

    def get_min(self):
        front_min = self.front_stack.get_min()
        back_min = self.back_stack.get_min()

        if front_min is None:
            return back_min
        if back_min is None:
            return front_min

        return min(front_min, back_min)

    def _rebalance(self):
        # Перекладываем элементы из back_stack в front_stack
        while self.back_stack.stack:
            self.front_stack.push(self.back_stack.pop())

    def _rebalance_reverse(self):
        # Перекладываем элементы из front_stack в back_stack
        while self.front_stack.stack:
            self.back_stack.push(self.front_stack.pop())
```

**Проблема с методом кредитов:**

Попытаемся применить метод кредитов для анализа амортизированной сложности операций в MinDeque.

Предположим, мы выдаем каждому элементу некоторое количество монет при добавлении в структуру. Проблема возникает при следующей последовательности операций:

1. Добавляем n элементов через push_back (все элементы попадают в back_stack)
2. Выполняем pop_front, что вызывает _rebalance и перекладывает все n элементов из back_stack в front_stack
3. Добавляем n новых элементов через push_back
4. Выполняем pop_back, что вызывает _rebalance_reverse и перекладывает все n элементов из front_stack в back_stack
5. Повторяем шаги 2-4 многократно

При каждом перекладывании элементов из одного стека в другой, мы выполняем O(n) операций. Если мы выдаем каждому элементу константное количество монет при добавлении, этих монет не хватит для оплаты многократных перекладываний.

**Теорема:** Не существует схемы кредитования с константным числом монет на элемент, которая обеспечивала бы амортизированную сложность O(1) для всех операций MinDeque.

**Доказательство:**

Рассмотрим последовательность из k циклов, где каждый цикл состоит из:
1. n операций pop_front (вызывающих _rebalance)
2. n операций pop_back (вызывающих _rebalance_reverse)

Каждый элемент может быть переложен из одного стека в другой до k раз. Если мы выдаем каждому элементу c монет при добавлении, то для k > c эти монеты не смогут оплатить все перекладывания.

Фактическая стоимость этой последовательности операций: O(k·n²)
Если бы амортизированная стоимость была O(1) на операцию, то общая амортизированная стоимость была бы O(k·n)

Поскольку k может быть произвольно большим, метод кредитов с константным числом монет на элемент не работает для этой структуры данных.

Это демонстрирует ограничение метода кредитов и мотивирует использование метода потенциалов, который мы рассмотрим далее.

## Метод потенциалов

### Формальная постановка метода потенциалов

Метод потенциалов — это мощный и элегантный способ анализа амортизированной сложности, который преодолевает ограничения метода кредитов. Он основан на введении функции потенциала, которая отражает "энергию" или "запас работы" структуры данных в каждом состоянии.

**Определение:** Пусть $D$ — структура данных с множеством состояний $S$. Функция потенциала $\Phi: S \rightarrow \mathbb{R}$ сопоставляет каждому состоянию $s \in S$ некоторое вещественное число, называемое потенциалом состояния.

Для последовательности операций $\sigma = (o_1, o_2, \ldots, o_n)$, начинающейся с состояния $s_0$ и порождающей последовательность состояний $s_1, s_2, \ldots, s_n$, амортизированная стоимость операции $o_i$ определяется как:

$$\hat{T}(o_i, s_{i-1}) = T(o_i, s_{i-1}) + \Phi(s_i) - \Phi(s_{i-1})$$

где $T(o_i, s_{i-1})$ — фактическая стоимость операции $o_i$ в состоянии $s_{i-1}$, а $\Phi(s_i) - \Phi(s_{i-1})$ — изменение потенциала.

**Теорема:** Суммарная амортизированная стоимость последовательности операций $\sigma$ равна:

$$\hat{T}(\sigma) = \sum_{i=1}^{n} \hat{T}(o_i, s_{i-1}) = \sum_{i=1}^{n} T(o_i, s_{i-1}) + \Phi(s_n) - \Phi(s_0)$$

**Доказательство:**

$$\begin{align*}
\hat{T}(\sigma) &= \sum_{i=1}^{n} \hat{T}(o_i, s_{i-1}) \\
&= \sum_{i=1}^{n} \left( T(o_i, s_{i-1}) + \Phi(s_i) - \Phi(s_{i-1}) \right) \\
&= \sum_{i=1}^{n} T(o_i, s_{i-1}) + \sum_{i=1}^{n} \left( \Phi(s_i) - \Phi(s_{i-1}) \right) \\
\end{align*}$$

Сумма $\sum_{i=1}^{n} \left( \Phi(s_i) - \Phi(s_{i-1}) \right)$ представляет собой телескопическую сумму:

$$\begin{align*}
\sum_{i=1}^{n} \left( \Phi(s_i) - \Phi(s_{i-1}) \right) &= (\Phi(s_1) - \Phi(s_0)) + (\Phi(s_2) - \Phi(s_1)) + \ldots + (\Phi(s_n) - \Phi(s_{n-1})) \\
&= \Phi(s_n) - \Phi(s_0)
\end{align*}$$

Таким образом:

$$\hat{T}(\sigma) = \sum_{i=1}^{n} T(o_i, s_{i-1}) + \Phi(s_n) - \Phi(s_0)$$

Если функция потенциала выбрана так, что $\Phi(s_0) \leq \Phi(s_n)$ или $\Phi(s_0)$ ограничено константой, то $\hat{T}(\sigma) \geq \sum_{i=1}^{n} T(o_i, s_{i-1})$, что соответствует определению амортизированной стоимости.

**Важные свойства функции потенциала:**

1. **Неотрицательность**: Обычно требуется, чтобы $\Phi(s) \geq \Phi(s_0)$ для всех достижимых состояний $s$. Это гарантирует, что амортизированная стоимость не меньше фактической.

2. **Отражение сложности**: Функция потенциала должна отражать "сложность" или "неэффективность" текущего состояния структуры данных.

3. **Компенсация дорогих операций**: Уменьшение потенциала при выполнении дорогой операции компенсирует её высокую фактическую стоимость.

**Связь с методом кредитов:**

Метод потенциалов можно рассматривать как обобщение метода кредитов. Если в методе кредитов монеты "привязаны" к элементам структуры данных, то в методе потенциалов они "привязаны" к состоянию структуры в целом. Это позволяет более гибко распределять "кредиты" между операциями.

Выбор подходящей функции потенциала — ключевой и часто нетривиальный шаг в применении метода потенциалов. В следующем разделе мы рассмотрим, как применить этот метод к структуре данных MinDeque, для которой метод кредитов оказался неприменим.

### Анализ MinDeque с помощью метода потенциалов

Вернемся к структуре данных MinDeque, для которой мы показали, что метод кредитов с константным числом монет на элемент не работает. Покажем, как с помощью метода потенциалов можно получить амортизированную оценку $O(1)$ для всех операций.

**Выбор функции потенциала:**

Определим функцию потенциала для MinDeque следующим образом:

$$\Phi(s) = c \cdot \left| |F| - |B| \right|$$

где $|F|$ — размер front_stack, $|B|$ — размер back_stack, а $c$ — некоторая положительная константа.

Эта функция потенциала отражает "дисбаланс" между двумя стеками. Чем больше разница в их размерах, тем выше потенциал, и тем "дороже" будет стоить операция, которая увеличивает этот дисбаланс.

**Анализ операций:**

1. **push_front(x):**
   - Фактическая стоимость: $T(\text{push\_front}, s) = O(1)$
   - Изменение состояния: $|F| \rightarrow |F| + 1$, $|B|$ не меняется
   - Изменение потенциала: $\Delta\Phi = c \cdot (|(|F| + 1) - |B|| - ||F| - |B||)$

   Если $|F| \geq |B|$, то $\Delta\Phi = c$
   Если $|F| < |B|$, то $\Delta\Phi = -c$

   В любом случае $|\Delta\Phi| = c = O(1)$

   - Амортизированная стоимость: $\hat{T}(\text{push\_front}, s) = O(1) + O(1) = O(1)$

2. **push_back(x):** Аналогично push_front, амортизированная стоимость $O(1)$.

3. **pop_front()** (когда front_stack не пуст):
   - Фактическая стоимость: $T(\text{pop\_front}, s) = O(1)$
   - Изменение состояния: $|F| \rightarrow |F| - 1$, $|B|$ не меняется
   - Изменение потенциала: $\Delta\Phi = c \cdot (|(|F| - 1) - |B|| - ||F| - |B||)$

   Если $|F| > |B|$, то $\Delta\Phi = -c$
   Если $|F| \leq |B|$, то $\Delta\Phi = c$

   В любом случае $|\Delta\Phi| = c = O(1)$

   - Амортизированная стоимость: $\hat{T}(\text{pop\_front}, s) = O(1) + O(1) = O(1)$

4. **pop_back()** (когда back_stack не пуст): Аналогично pop_front, амортизированная стоимость $O(1)$.

5. **pop_front()** (когда front_stack пуст, вызывается _rebalance):
   - Фактическая стоимость: $T(\text{pop\_front}, s) = O(|B|)$ (перекладываем все элементы из back_stack)
   - Изменение состояния: $|F| = 0 \rightarrow |B| - 1$, $|B| \rightarrow 0$
   - Изменение потенциала: $\Delta\Phi = c \cdot (|(|B| - 1) - 0| - |0 - |B||) = 0$

   - Амортизированная стоимость: $\hat{T}(\text{pop\_front}, s) = O(|B|) + 0 = O(|B|)$

   Это выглядит плохо, но давайте рассмотрим, что происходило до этой операции. Чтобы front_stack стал пустым, мы должны были выполнить $|F|$ операций pop_front, каждая с амортизированной стоимостью $O(1)$. При этом потенциал уменьшался на $c$ при каждой операции, в сумме на $c \cdot |F|$.

   Если изначально $|F| \approx |B|$, то накопленное уменьшение потенциала составляет $c \cdot |B|$, что компенсирует высокую стоимость _rebalance.

6. **pop_back()** (когда back_stack пуст): Аналогично случаю 5, амортизированная стоимость в среднем $O(1)$.

**Теорема:** При использовании функции потенциала $\Phi(s) = c \cdot \left| |F| - |B| \right|$ с достаточно большой константой $c$, амортизированная стоимость всех операций в MinDeque составляет $O(1)$.

**Доказательство:**

Мы уже показали, что для простых операций (без _rebalance) амортизированная стоимость равна $O(1)$.

Для операций с _rebalance, хотя их фактическая стоимость может быть $O(n)$, они компенсируются накопленным уменьшением потенциала от предыдущих операций.

Более формально, рассмотрим последовательность операций между двумя последовательными _rebalance. Пусть изначально $|F| \approx |B| \approx n/2$. После $n/2$ операций pop_front, front_stack становится пустым, и выполняется _rebalance. Суммарное уменьшение потенциала за эти $n/2$ операций составляет $c \cdot n/2$, что при $c \geq 2$ компенсирует стоимость _rebalance, равную $O(n)$.

Таким образом, амортизированная стоимость любой последовательности операций с MinDeque составляет $O(1)$ на операцию.

**Пример:**

```python
# Пример использования MinDeque с анализом потенциалов
deque = MinDeque()

# Начальное состояние: |F| = 0, |B| = 0, Φ = 0

# Добавляем n элементов через push_back
for i in range(n):
    deque.push_back(i)
    # После i-й операции: |F| = 0, |B| = i, Φ = c·i

# Состояние: |F| = 0, |B| = n, Φ = c·n

# Выполняем pop_front, что вызывает _rebalance
# Фактическая стоимость: O(n)
# После операции: |F| = n-1, |B| = 0, Φ = c·(n-1)
# Изменение потенциала: Δ = c·(n-1) - c·n = -c
# Амортизированная стоимость: O(n) - c = O(1) при c = O(n)
value = deque.pop_front()
```

Этот пример показывает, как метод потенциалов позволяет получить амортизированную оценку $O(1)$ для операций MinDeque, в то время как метод кредитов с константным числом монет на элемент не работает для этой структуры данных.

### Эквивалентность методов амортизационного анализа

Хотя мы рассмотрели три различных метода амортизационного анализа (агрегационный, кредитов и потенциалов), на самом деле они эквивалентны по своей выразительной силе. В этом разделе мы покажем, как результаты, полученные агрегационным методом или методом кредитов, можно переформулировать в терминах метода потенциалов.

#### Формализация агрегационного метода через потенциал

**Теорема:** Любой результат, полученный агрегационным методом, можно формализовать через некоторую функцию потенциала.

**Доказательство:**

Пусть с помощью агрегационного метода мы получили, что последовательность операций $\sigma = (o_1, o_2, \ldots, o_n)$ имеет суммарную стоимость $T(\sigma) = O(f(n))$, где $f(n)$ — некоторая функция от длины последовательности.

Определим функцию потенциала $\Phi(s_i)$ как сумму фактических стоимостей всех операций от начального состояния до состояния $s_i$, минус сумма амортизированных стоимостей этих операций:

$$\Phi(s_i) = \sum_{j=1}^{i} T(o_j, s_{j-1}) - \sum_{j=1}^{i} \hat{T}(o_j)$$

где $\hat{T}(o_j) = \frac{f(n)}{n}$ — амортизированная стоимость операции, одинаковая для всех операций в агрегационном методе.

Тогда амортизированная стоимость операции $o_i$ в методе потенциалов:

$$\hat{T}(o_i, s_{i-1}) = T(o_i, s_{i-1}) + \Phi(s_i) - \Phi(s_{i-1})$$

$$= T(o_i, s_{i-1}) + \left(\sum_{j=1}^{i} T(o_j, s_{j-1}) - \sum_{j=1}^{i} \hat{T}(o_j)\right) - \left(\sum_{j=1}^{i-1} T(o_j, s_{j-1}) - \sum_{j=1}^{i-1} \hat{T}(o_j)\right)$$

$$= T(o_i, s_{i-1}) + T(o_i, s_{i-1}) - \hat{T}(o_i) = \hat{T}(o_i)$$

Таким образом, мы получили ту же амортизированную стоимость, что и в агрегационном методе.

Для корректности метода потенциалов также требуется, чтобы $\Phi(s_0) \leq \Phi(s_n)$ или $\Phi(s_0)$ было ограничено константой. Поскольку $\Phi(s_0) = 0$ и $\Phi(s_n) = T(\sigma) - n \cdot \frac{f(n)}{n} = T(\sigma) - f(n) \leq 0$ (так как $T(\sigma) \leq f(n)$ по предположению агрегационного метода), мы можем добавить константу $C$ ко всем значениям потенциала, чтобы обеспечить неотрицательность.

#### Формализация метода кредитов через потенциал

**Теорема:** Любой результат, полученный методом кредитов, можно формализовать через некоторую функцию потенциала.

**Доказательство:**

В методе кредитов мы выдаем монеты при выполнении операций и используем их для оплаты будущих операций. Определим функцию потенциала $\Phi(s_i)$ как общее количество неиспользованных монет (кредитов) в состоянии $s_i$.

Пусть $c_i$ — количество монет, выданных при выполнении операции $o_i$, а $p_i$ — количество монет, использованных для оплаты операции $o_i$. Тогда:

$$\Phi(s_i) - \Phi(s_{i-1}) = c_i - p_i$$

Амортизированная стоимость операции $o_i$ в методе кредитов равна фактической стоимости плюс количество выданных монет:

$$\hat{T}(o_i, s_{i-1}) = T(o_i, s_{i-1}) + c_i$$

В методе потенциалов амортизированная стоимость:

$$\hat{T}(o_i, s_{i-1}) = T(o_i, s_{i-1}) + \Phi(s_i) - \Phi(s_{i-1}) = T(o_i, s_{i-1}) + c_i - p_i$$

Поскольку $p_i$ монет используются для оплаты части фактической стоимости $T(o_i, s_{i-1})$, можно записать $T(o_i, s_{i-1}) = p_i + r_i$, где $r_i$ — оставшаяся часть стоимости. Тогда:

$$\hat{T}(o_i, s_{i-1}) = p_i + r_i + c_i - p_i = r_i + c_i$$

Это в точности соответствует амортизированной стоимости в методе кредитов, если учесть, что $r_i$ — это часть фактической стоимости, не оплаченная монетами.

Функция потенциала $\Phi(s)$ неотрицательна по определению (количество неиспользованных монет не может быть отрицательным), что удовлетворяет требованиям метода потенциалов.

#### Заключение

Таким образом, мы показали, что все три метода амортизационного анализа (агрегационный, кредитов и потенциалов) эквивалентны по своей выразительной силе. Метод потенциалов является наиболее общим и гибким, позволяя формализовать результаты, полученные другими методами.

Выбор конкретного метода зависит от удобства его применения к конкретной задаче и структуре данных. Агрегационный метод прост, но не всегда позволяет получить точные оценки. Метод кредитов интуитивно понятен, но может быть неприменим в некоторых случаях (как мы видели на примере MinDeque). Метод потенциалов наиболее мощный, но требует нетривиального выбора функции потенциала.

## From gpt

https://chatgpt.com/share/68145fe1-b298-800d-8a6c-d9084f150b3c

https://alvmer.github.io/complexity/amortized.html

https://habr.com/ru/articles/208624/

https://neerc.ifmo.ru/wiki/index.php?title=%D0%90%D0%BC%D0%BE%D1%80%D1%82%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7#.D0.9C.D0.B5.D1.82.D0.BE.D0.B4_.D1.83.D1.81.D1.80.D0.B5.D0.B4.D0.BD.D0.B5.D0.BD.D0.B8.D1.8F

## Классические примеры

### Пример 1. Динамический массив

* **Состояние**: текущий размер массива $n$ и ёмкость $m$.
* **Потенциал**: $\Phi = 2n - m$ (неотрицателен после каждой расширения).
* **Анализ**:

  * Обычное `push`: $c=1$, изменение потенциала $\Delta\Phi = 2$.
    $\hat c = 1 + 2 = 3$.
  * При расширении: фактический $c = n+1$ (копирования + вставка),
    $\Delta\Phi = (2n - 2m) - (2n - m) = m - 2m = -m$.
    Поскольку $m=n$ до расширения, $\hat c = (n+1) - n =1$.
* **Заключение**: амортизационная стоимость операции — $O(1)$.

### Пример 2. Стек с мульти-попом

* Операции: `push` — $c=1$, `pop` — $c=1$, `multipop(k)` — $c=\min(k,h)$, где $h$ — текущая высота.
* **Агрегированный метод**: за любые $n$ операций `push` и `pop` реально выполняется не более $n$ `push` и $n$ `pop`, значит суммарно $O(n)$.
  Следовательно, амортизационная стоимость каждой — $O(1)$.

### Пример 3. DSU (с Union by Rank + Path Compression)

* Показывает амортизацию $\alpha(n)$ (обратная аккумулированная логарифмическая функция).
* Формальный анализ опирается на сложные потенциалы, задаваемые рангами и глубинами, и приводит к оценке $O(\alpha(n))$ на операцию.

---

## Вывод

Амортизационный анализ обеспечивает **строгие** детерминированные оценки среднего поведения последовательности операций без предположений о распределении входных данных. Три основных метода (агрегированный, счётов и потенциальный) эквивалентны по своей вычислительной силе и позволяют выбирать наиболее удобный способ доказательства. Примеры динамического массива, стека с мульти-попом и DSU иллюстрируют мощь этого подхода в проектировании высокопроизводительных структур данных.